# Adding Tool?

In [1]:
import os
with open("/home/billxbf/Documents/myks/openai.key", "r") as f:
    keys = f.readlines()
    os.environ["OPENAI_API_KEY"] = keys[0].strip()
with open("/home/billxbf/Documents/myks/serpapi.key", "r") as f:
    keys = f.readlines()
    os.environ["SERPAPI_API_KEY"] = keys[0].strip()

In [2]:
from utils.DataLoader import DataLoader
from utils.Evaluator import Evaluator
from algos.PWS import *
from algos.react import ReactBase
from algos.notool import IO, CoT
from prompts import fewshots

In [3]:
def save_data(dataset, data, save_path):
    dataset["preds"] = data["preds"]
    dataset["em"]  = data["em"]
    dataset["f1"] = data["f1"]
    dataset["acc"] = data["acc"]
    dataset["wall_time"] = data["wall_time"]
    dataset["total_tokens"] = data["total_tokens"]
    dataset["steps"] = data["steps"]
    dataset["tool_cost"] = data["tool_cost"]
    dataset["token_cost"] = data["token_cost"]
    dataset["total_cost"] = data["total_cost"]
    if "planner_log" in data:
        dataset["planner_log"] = data["planner_log"]
    if "solver_log" in data:
        dataset["solver_log"] = data["solver_log"]
    dataset.to_csv(save_path, index=False)
    return dataset

In [4]:
EVAL_LLM = "gpt-3.5-turbo"
EVAL_DATASET = "hotpot_qa"
SEED = 2024
SAVE_DIR = "results/hotpot_qa_{}_{}.csv"

## REACT

In [12]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(50)
react = ReactBase(fewshot=fewshots.HOTPOTQA_REACT, model_name=EVAL_LLM, max_iter=8, verbose=False)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=react)

Found cached dataset hotpot_qa (/home/billxbf/workspace/PWS/data/hotpot_qa/hotpot_qa/fullwiki/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
response, data = eval.run()
df = save_data(dataset, data, SAVE_DIR.format("react", EVAL_LLM))
response


******************* Start Evaluation *******************



 10%|█         | 5/50 [02:27<21:38, 28.86s/it]

In [9]:
df.head(10)

,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,In what city and state was the vocalist for E...,"Redwood City, California",nan,False,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
1,"What was the birth date of the director of ""Th...","December 28, 1888March",nan,False,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
2,What is the name of the cognac house that make...,Jas Hennessy & Co.,Hennessy,False,0.500000,1,8.891754,5092.0,3.0,0.0,0.010184,0.010184
3,The Wisconsin Badgers football team has had a ...,The Iron Horse,the Iron Horse or the Horse,False,0.666667,0,12.183595,7105.0,4.0,0.0,0.014210,0.014210
4,"Which film director is older, Jan Troell or Ag...",Agnès Varda,Jan Troell,False,0.000000,0,9.318587,5263.0,3.0,0.0,0.010526,0.010526
5,"Foshan Lingnan Mingzhu Gymnasium, where Boxing...",2006,2006,True,1.000000,1,7.464730,3284.0,2.0,0.0,0.006568,0.006568
6,In between Parenting and Cook's Illustrated wh...,Parenting,Parenting,True,1.000000,1,10.895709,7210.0,4.0,0.0,0.014420,0.014420
7,"In the year 2016, how many passengers used the...",11.04 million,Agent stopped due to iteration limit or time l...,False,0.000000,0,33.010987,21835.0,9.0,0.0,0.043670,0.043670
8,What was the nationality of Robert Kardashian'...,American,Agent stopped due to iteration limit or time l...,False,0.000000,0,21.680504,14876.0,9.0,0.0,0.029752,0.029752
9,In what Ohio city does the highway which the n...,Cincinnati,Agent stopped due to iteration limit or time l...,False,0.000000,0,30.278475,17950.0,9.0,0.0,0.035900,0.035900


## PWSBase

In [5]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(20)
pwsbase = PWS_Base(fewshot=fewshots.HOTPOTQA_PWS_BASE, planner_model=EVAL_LLM, solver_model=EVAL_LLM, available_tools=["Wikipedia","LLM","Google"])
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=pwsbase)

Found cached dataset hotpot_qa (/home/billxbf/workspace/PWS/data/hotpot_qa/hotpot_qa/fullwiki/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
response, data = eval.run()
#df = save_data(dataset, data, SAVE_DIR.format("pws", EVAL_LLM))
response


******************* Start Evaluation *******************



 15%|█▌        | 3/20 [00:56<05:08, 18.12s/it]/home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████| 20/20 [06:55<00:00, 20.76s/it]


{'avg_em': 0.4,
 'avg_f1': 0.41,
 'avg_acc': 0.4,
 'avg_wall_time': 16.947114300727844,
 'avg_total_tokens': 1413.6,
 'avg_total_cost': 0.018396000000000003,
 'avg_steps': 4.85,
 'avg_token_cost': 0.007896,
 'avg_tool_cost': 0.010499999999999999}

In [12]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(100)
pwsbase = PWS_Base(fewshot=fewshots.HOTPOTQA_PWS_BASE, planner_model=EVAL_LLM, solver_model=EVAL_LLM, available_tools=["Wikipedia","LLM","Google", "WolframAlpha","Calculator"])
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=pwsbase)

Found cached dataset hotpot_qa (/home/billxbf/workspace/PWS/data/hotpot_qa/hotpot_qa/fullwiki/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
response, data = eval.run()
df = save_data(dataset, data, SAVE_DIR.format("pws_test", EVAL_LLM))
response


******************* Start Evaluation *******************



  3%|▎         | 3/100 [01:05<33:45, 20.88s/it]/home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████| 100/100 [1:20:49<00:00, 48.49s/it]


ValueError: Length of values (99) does not match length of index (100)

In [14]:
response

{'avg_em': 0.3,
 'avg_f1': 0.38843368248631405,
 'avg_acc': 0.4,
 'avg_wall_time': 33.28135439603015,
 'avg_total_tokens': 1588.888888888889,
 'avg_total_cost': 0.01794442424242424,
 'avg_steps': 4.898989898989899,
 'avg_token_cost': 0.00966159595959596,
 'avg_tool_cost': 0.008282828282828282}

In [15]:
data

{'label': ['The Cripple of Inishmaan',
  'Henry Beaufort',
  'Rubeus Hagrid',
  'Trip hop',
  'Mark Strong',
  'Kirk Ferentz',
  'Adobe Systems',
  'Marta Eggerth',
  'Teucrium',
  '3 November 1949',
  'David Cameron',
  'Providence College',
  'Bee Gees',
  'Tanner Foust',
  'his musical composition',
  'A Holly Jolly Christmas',
  'England',
  'The Shins',
  'Danny Kanell',
  'Robert Pattinson',
  'What Happens in Vegas',
  'no',
  'Jimmie Davis',
  '2012 Formula One season',
  'Alakozai',
  'no',
  '1892',
  'Samoa Time',
  'John Malcolm Fraser',
  'Mauro Tassotti',
  'English',
  'United States',
  'Park Jin-young',
  'United States',
  'the 1981 season',
  'Atlantic Ocean',
  'Clarke',
  'director',
  'a transcontinental flight',
  'Nassau County',
  'National Autonomous University of Mexico',
  'A-ha',
  'no',
  'Valencia, Santa Clarita, California',
  'National Football League',
  'Lancashire County Football Association Cup',
  'Northern Irish',
  'Altice USA',
  'cobbled',
  'M

In [19]:
df = save_data(dataset, data, SAVE_DIR.format("pws_test", EVAL_LLM))

In [23]:
res = pwsbase.run(dataset["question"][0])

In [28]:
print(res["planner_log"])

For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...) 

Tools can be one of the following:
Wikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.
LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.
Google[input]: Worker that searches results from Google. Useful when you need to find short and succinct answers about a specific topic. Input should be a search q

In [8]:
df

,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost,planner_log,solver_log
0,Noble is a 2014 film written and directed by S...,The Cripple of Inishmaan,There is no information provided about what Sa...,False,0.0,0,17.495597,1788,5,0.00,0.007266,0.007266,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
1,Who was the English clergyman and Archbishop o...,Henry Beaufort,There is no evidence that Otto von Ziegenhain ...,False,0.0,0,23.318231,1503,5,0.00,0.008046,0.008046,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
2,Which fictional character in the Harry Potter ...,Rubeus Hagrid,Rubeus Hagrid.,True,1.0,1,9.478219,1695,3,0.00,0.011778,0.011778,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
3,What Classification is the album by UK band Se...,Trip hop,Alternative Rock.,False,0.0,0,14.910080,1434,5,0.00,0.003660,0.003660,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
4,What actor from The Imitation Game also starre...,Mark Strong,Benedict Cumberbatch.,False,0.0,0,24.711618,1779,5,0.01,0.011190,0.021190,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
5,This 2006 Iowa Hawkeyes football coach played ...,Kirk Ferentz,Kirk Ferentz.,True,1.0,1,28.668656,2601,7,0.00,0.020448,0.020448,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
6,Ikivo Animator integrates with a software appl...,Adobe Systems,Ikivo.,False,0.0,0,16.383253,1374,4,0.01,0.007770,0.017770,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
7,What American actress born in 1912 did the vio...,Marta Eggerth,Lillian Gish.,False,0.0,0,25.082648,1303,5,0.00,0.005522,0.005522,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
8,"Which plant has more species, Teucrium or Atal...",Teucrium,Teucrium.,True,1.0,1,14.217040,1641,4,0.00,0.010914,0.010914,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...
9,When was the British-American journalist and e...,3 November 1949,"The British-American journalist and editor, Gr...",False,0.0,0,21.869306,1570,5,0.00,0.009584,0.009584,"For the following tasks, make plans that can s...",Solve the following task or problem. To assist...


In [10]:
print(df["planner_log"][4])

For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...) 

Tools can be one of the following:
Wikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.
LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.
Google[input]: Worker that searches results from Google. Useful when you need to find short and succinct answers about a specific topic. Input should be a search q

In [11]:
print(df["solver_log"][4])

Solve the following task or problem. To assist you, we provide some plans and corresponding evidences that might be helpful. Notice that some of these information contain noise so you should trust them with caution.

What actor from The Imitation Game also starred in "Revolver", "RocknRolla" and "Sherlock Holmes"?
Plan: Search for the cast of The Imitation Game.
Evidence:
In 1939, newly created British intelligence agency MI6 recruits Cambridge mathematics alumnus Alan Turing (Benedict Cumberbatch) to crack Nazi codes, including Enigma -- which cryptanalysts had thought unbreakable. Turing's team, including Joan Clarke (Keira Knightley), analyze Enigma messages while he builds a machine to decipher them. Turing and team finally succeed and become heroes, but in 1952, the quiet genius encounters disgrace when authorities reveal he is gay and send him to prison.… MORE
Plan: Look for the name of the actor who starred in "Revolver", "RocknRolla" and "Sherlock Holmes".
Evidence:
Benedict Cu